In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import hamming_loss, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import os
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_val_predict
from sklearn.metrics import classification_report, hamming_loss
import numpy as np

In [ ]:
def merge_csv_files(folder_path):
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    dataframes = [pd.read_csv(file) for file in all_files]
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

In [ ]:
labelled_folder = r"C:\github\news\news\german-news\preprocess data\data_ready_for_analysis\labelled"

In [ ]:
labelled_data = merge_csv_files(labelled_folder)

In [ ]:
labelled_data

In [ ]:
validation_data = labelled_data.sample(n=30, random_state=42)
remaining_data = labelled_data.drop(validation_data.index)

In [ ]:
labels = ['Fake News', 'Extreme bias', 'clickbait', 'credible']
validation_data[labels].sum()

In [ ]:
labels = ['Fake News', 'Extreme bias', 'clickbait', 'credible']
remaining_data[labels].sum()

In [ ]:
unlabelled_folder = r"C:\github\news\news\german-news\preprocess data\data_ready_for_analysis\unlabelled"

In [ ]:
unlabelled_data = merge_csv_files(unlabelled_folder)

In [ ]:
label_mapping = {
    "Fake News": 1,
    "Extreme bias": 2,
    "clickbait": 3,
    "credible": 0
}

def map_labels(row):
    labels = [label_mapping[label] for label in label_mapping if row[label] == 1]
    return "-".join(map(str, sorted(labels))) if labels else "none"

remaining_data["label"] = remaining_data.apply(map_labels, axis=1)
remaining_data

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
remaining_data['label']= le.fit_transform(remaining_data['label']) 
remaining_data

In [ ]:
X = remaining_data[['title', 'description', 'body']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
y = remaining_data[['label']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
unique_labels, counts = np.unique(y, return_counts=True)

# Print unique labels and their counts
for label, count in zip(unique_labels, counts):
    print(f"Label: {label}, Count: {count}")

In [ ]:
print("Unique classes after encoding:", np.unique(y_train))

In [ ]:
print("Unique classes after encoding:", np.unique(y_test))

In [ ]:
vectorizer = TfidfVectorizer(max_features=3000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
# Import necessary libraries
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# Initialize dictionaries to store results
results = {}

# Fit and evaluate GradientBoostingClassifier
gb_model = MultiOutputClassifier(GradientBoostingClassifier(random_state=42))
gb_model.fit(X_train_tfidf, y_train)
gb_pred = gb_model.predict(X_test_tfidf)
gb_acc = accuracy_score(y_test, gb_pred)
results['MultiOutputClassifier'] = gb_acc
print(f"Accuracy with MultiOutputClassifier: {gb_acc}")

# Fit and evaluate XGBoost
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train_tfidf, y_train)
xgb_pred = xgb_model.predict(X_test_tfidf)
xgb_acc = accuracy_score(y_test, xgb_pred)
results['XGBoost'] = xgb_acc
print(f"Accuracy with XGBoost: {xgb_acc}")

# Fit and evaluate LightGBM
lgbm_model = LGBMClassifier(random_state=42)
lgbm_model.fit(X_train_tfidf, y_train)
lgbm_pred = lgbm_model.predict(X_test_tfidf)
lgbm_acc = accuracy_score(y_test, lgbm_pred)
results['LightGBM'] = lgbm_acc
print(f"Accuracy with LightGBM: {lgbm_acc}")

# Fit and evaluate CatBoost
catboost_model = CatBoostClassifier(random_state=42, verbose=0)
catboost_model.fit(X_train_tfidf, y_train)
catboost_pred = catboost_model.predict(X_test_tfidf)
catboost_acc = accuracy_score(y_test, catboost_pred)
results['CatBoost'] = catboost_acc
print(f"Accuracy with CatBoost: {catboost_acc}")

# Fit and evaluate BaggingClassifier with DecisionTreeClassifier as base estimator
from sklearn.tree import DecisionTreeClassifier

bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42, n_estimators=100)
bagging_model.fit(X_train_tfidf, y_train)
bagging_pred = bagging_model.predict(X_test_tfidf)
bagging_acc = accuracy_score(y_test, bagging_pred)
results['BaggingClassifier_DecisionTree'] = bagging_acc
print(f"Accuracy with BaggingClassifier (DecisionTree): {bagging_acc}")

# Optional: Evaluate BaggingClassifier with other base estimators
# Uncomment the following to try KNeighborsClassifier or LogisticRegression as base estimators.

# from sklearn.neighbors import KNeighborsClassifier
# bagging_knn = BaggingClassifier(base_estimator=KNeighborsClassifier(), random_state=42, n_estimators=100)
# bagging_knn.fit(X_train_tfidf, y_train)
# bagging_knn_pred = bagging_knn.predict(X_test_tfidf)
# bagging_knn_acc = accuracy_score(y_test, bagging_knn_pred)
# results['BaggingClassifier_KNeighbors'] = bagging_knn_acc
# print(f"Accuracy with BaggingClassifier (KNeighbors): {bagging_knn_acc}")

# from sklearn.linear_model import LogisticRegression
# bagging_lr = BaggingClassifier(base_estimator=LogisticRegression(), random_state=42, n_estimators=100)
# bagging_lr.fit(X_train_tfidf, y_train)
# bagging_lr_pred = bagging_lr.predict(X_test_tfidf)
# bagging_lr_acc = accuracy_score(y_test, bagging_lr_pred)
# results['BaggingClassifier_LogisticRegression'] = bagging_lr_acc
# print(f"Accuracy with BaggingClassifier (LogisticRegression): {bagging_lr_acc}")

# Display all results
print("\nSummary of Model Accuracies:")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy:.4f}")


In [ ]:
# Import necessary libraries
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# Initialize dictionaries to store results
results = {}

# Fit and evaluate GradientBoostingClassifier
# gb_model = GradientBoostingClassifier(random_state=42)
# gb_model.fit(X_train_tfidf, y_train)
# gb_pred = gb_model.predict(X_test_tfidf)
# gb_acc = accuracy_score(y_test, gb_pred)
# results['GradientBoostingClassifier'] = gb_acc
# print(f"Accuracy with GradientBoostingClassifier: {gb_acc}")

# # Fit and evaluate XGBoost
# xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
# xgb_model.fit(X_train_tfidf, y_train)
# xgb_pred = xgb_model.predict(X_test_tfidf)
# xgb_acc = accuracy_score(y_test, xgb_pred)
# results['XGBoost'] = xgb_acc
# print(f"Accuracy with XGBoost: {xgb_acc}")

# # Fit and evaluate LightGBM
# lgbm_model = LGBMClassifier(random_state=42)
# lgbm_model.fit(X_train_tfidf, y_train)
# lgbm_pred = lgbm_model.predict(X_test_tfidf)
# lgbm_acc = accuracy_score(y_test, lgbm_pred)
# results['LightGBM'] = lgbm_acc
# print(f"Accuracy with LightGBM: {lgbm_acc}")

# # Fit and evaluate CatBoost
# catboost_model = CatBoostClassifier(random_state=42, verbose=0)
# catboost_model.fit(X_train_tfidf, y_train)
# catboost_pred = catboost_model.predict(X_test_tfidf)
# catboost_acc = accuracy_score(y_test, catboost_pred)
# results['CatBoost'] = catboost_acc
# print(f"Accuracy with CatBoost: {catboost_acc}")

# Fit and evaluate BaggingClassifier with DecisionTreeClassifier as base estimator
from sklearn.tree import DecisionTreeClassifier

bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42, n_estimators=100)
bagging_model.fit(X_train_tfidf, y_train)
bagging_pred = bagging_model.predict(X_test_tfidf)
bagging_acc = accuracy_score(y_test, bagging_pred)
results['BaggingClassifier_DecisionTree'] = bagging_acc
print(f"Accuracy with BaggingClassifier (DecisionTree): {bagging_acc}")

# Optional: Evaluate BaggingClassifier with other base estimators
# Uncomment the following to try KNeighborsClassifier or LogisticRegression as base estimators.

from sklearn.neighbors import KNeighborsClassifier
bagging_knn = BaggingClassifier(estimator=KNeighborsClassifier(), random_state=42, n_estimators=100)
bagging_knn.fit(X_train_tfidf, y_train)
bagging_knn_pred = bagging_knn.predict(X_test_tfidf)
bagging_knn_acc = accuracy_score(y_test, bagging_knn_pred)
results['BaggingClassifier_KNeighbors'] = bagging_knn_acc
print(f"Accuracy with BaggingClassifier (KNeighbors): {bagging_knn_acc}")

from sklearn.linear_model import LogisticRegression
bagging_lr = BaggingClassifier(estimator=LogisticRegression(), random_state=42, n_estimators=100)
bagging_lr.fit(X_train_tfidf, y_train)
bagging_lr_pred = bagging_lr.predict(X_test_tfidf)
bagging_lr_acc = accuracy_score(y_test, bagging_lr_pred)
results['BaggingClassifier_LogisticRegression'] = bagging_lr_acc
print(f"Accuracy with BaggingClassifier (LogisticRegression): {bagging_lr_acc}")

# Display all results
print("\nSummary of Model Accuracies:")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy:.4f}")


In [ ]:
results = {}
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

# Define models
models = {
    "GradientBoostingClassifier": GradientBoostingClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(random_state=42, verbose=0),
    "BaggingClassifier_DecisionTree": BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42, n_estimators=100),
    "BaggingClassifier_KNeighbors": BaggingClassifier(estimator=KNeighborsClassifier(), random_state=42, n_estimators=100),
    "BaggingClassifier_LogisticRegression": BaggingClassifier(estimator=LogisticRegression(), random_state=42, n_estimators=100)
}

# Evaluate each model using cross-validation
for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    
    # Perform cross-validation
    scores = cross_val_score(model, X_train_tfidf, y_train, cv=kf, scoring='accuracy')
    results[model_name] = {
        "mean_accuracy": np.mean(scores),
        "cv_scores": scores
    }
    
    # Generate cross-validated predictions (for metrics like classification report)
    y_pred_cv = cross_val_predict(model, X_train_tfidf, y_train, cv=kf)
    print(f"Cross-Validated Accuracy for {model_name}: {np.mean(scores):.4f}")
    print("Classification Report:")
    print(classification_report(y_train, y_pred_cv))

# Display summarized results
print("\nSummary of Model Accuracies:")
for model_name, result in results.items():
    print(f"{model_name}: Mean Accuracy: {result['mean_accuracy']:.4f}")